# MSDS3436 Final Project
<h3>Face Mask Detection using Sage Maker</h3>

2022WI_MSDS_436-DL_SEC55 Analytics Systems Engineering

Mark Stockwell | Wesley Ng

<hr>

## AWS Setup

You will need to upload two files with no extension, "credentials" and "config".

The credentials file has your API key, config is preferred region

Put the following in each, change values to yours:
<hr>
credentials

```
[default]
aws_access_key_id = AKIAXN6R7JXYxxxxxxxx 
aws_secret_access_key = 1prI4W8J254zR5/ishEilmqskaZLOnazxxxxxxxx
```
<hr>
config

```
[default]
region = us-east-1
```

In [ ]:
!pip install boto3
!apt-get update
!pip3 install awscli
!pip3 install boto3==1.18
!pip3 install sagemaker

import boto3
from google.colab import files
files.upload()
! mkdir ~/.aws
! cp credentials ~/.aws/
! cp config ~/.aws/
! chmod 600 ~/.aws/credentials
! chmod 600 ~/.aws/config

# all ok? this command should work
comprehend = boto3.client(service_name='comprehend', region_name="us-east-1")
text = "Machine learning will automate jobs that most people thought could only be done by people." #~Dave Waters
print("text to perform sentiment analysis: \n",text)
comprehend.detect_sentiment(Text=text, LanguageCode='en')

     |████████████████████████████████| 132 kB 3.6 MB/s 
     |████████████████████████████████| 79 kB 5.8 MB/s 
     |████████████████████████████████| 8.6 MB 28.8 MB/s 
     |████████████████████████████████| 138 kB 32.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


## Tensorflow API setup

In [ ]:
#Required for Deploying Model to Local Tensorflow Server

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

#Installing Tensorflow Model Server
!apt-get install tensorflow-model-server

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  13947      0 --:--:-- --:--:-- --:--:-- 13882
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x

In [ ]:
import boto3
import os
import tensorflow as tf
import json
import pandas as pd
import requests
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

s3 = boto3.resource("s3")

model_path = "./"

#Downloading the Training Images 
bucket = s3.Bucket('msds436facemaskdetection')
for obj in bucket.objects.filter(Prefix = 'mask_model.zip'):
    print(obj.key)
    file_name = obj.key.split("/")[-1]
    file_path = os.path.join(model_path, file_name)
    print("Downloading to {}".format(file_path))
    bucket.download_file(obj.key, file_path)

!unzip mask_model.zip

In [ ]:
def download_s3_folder(bucket_name, s3_folder, local_dir=None):
    """
    Download the contents of a folder directory
    Args:
        bucket_name: the name of the s3 bucket
        s3_folder: the folder path in the s3 bucket
        local_dir: a relative or absolute directory path in the local file system
    """
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=s3_folder):
        target = obj.key if local_dir is None \
            else os.path.join(local_dir, os.path.relpath(obj.key, s3_folder))
        if not os.path.exists(os.path.dirname(target)):
            os.makedirs(os.path.dirname(target))
        if obj.key[-1] == '/':
            continue
        bucket.download_file(obj.key, target)

In [ ]:
#Download test with mask files
download_s3_folder("msds436facemaskdetection", "test_with_mask")

In [ ]:
#Download test without mask files
download_s3_folder("msds436facemaskdetection", "test_without_mask")

In [ ]:
#Create folder containing all files
if not os.path.exists("./deploy_test"):
  os.mkdir("./deploy_test/")
if not os.path.exists("./deploy_test/unknown"):
  os.mkdir("./deploy_test/unknown")

%cp -r ./test_with_mask/. ./deploy_test/unknown
%cp -r ./test_without_mask/. ./deploy_test/unknown

In [ ]:
#Strip out hidden files if exists
%rm -r ./test_with_mask/.ipynb_checkpoints
%rm -r ./test_without_mask/test/.ipynb_checkpoints

rm: cannot remove './test_with_mask/.ipynb_checkpoints': No such file or directory
rm: cannot remove './test_without_mask/test/.ipynb_checkpoints': No such file or directory


In [ ]:
#Test Paths
deploy_test_path = "./deploy_test"

In [ ]:
#Create the test dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    deploy_test_path, color_mode='rgb', batch_size=1, image_size=(256,
    256), shuffle=True, seed=None, validation_split=None, subset=None,
    interpolation='bilinear', follow_links=False,
    # crop_to_aspect_ratio=False
)

Found 10 files belonging to 1 classes.


In [ ]:
new_model = tf.keras.models.load_model('saved_model/mask_model')
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 50)                1638450   
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 16,353,189
Trainable params: 1,638,501
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
#Set up model in deployment path
export_path = "./DEPLOY/"
version = 1
export_path = os.path.join(export_path, str(version))

if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}


new_model.save(export_path, save_format="tf")

INFO:tensorflow:Assets written to: ./DEPLOY/1/assets


In [ ]:
os.environ["MODEL_DIR"] = "/content/DEPLOY"

In [ ]:
%pwd ./DEPLOY

'/content'

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8571 \
  --model_name=helloworld \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 0 in a separate thread.


In [ ]:
!tail server.log

In [ ]:
deploy_request = pd.Series(list(test_ds.as_numpy_iterator())).to_json(orient='values')
data = json.dumps({"signature_name": "serving_default", "instances": deploy_request})
# print(data)

In [ ]:
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8571/v1/models/helloworld:predict', data=data, headers=headers)

print(json_response.text)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
with open('json_data.json', 'w') as outfile:
    outfile.write(json_response.text)

In [ ]:
print(type(json_response.text))

<class 'str'>


In [ ]:
img_path = './deploy_test/unknown/seed9998.png'
img_path = './deploy_test/unknown/with-mask-default-mask-seed9995.png'
img = image.load_img(img_path, target_size=(256, 256))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
new_model.predict(x)

array([[0.]], dtype=float32)

In [ ]:
def run_prediction(data,headers,endpoint):
    json_response = requests.post(endpoint,data=data,headers=headers)
    print(json_response.text)
    predictions = json.loads(json_response.text)
    return predictions

In [ ]:
endpoint = 'http://localhost:8571/v1/models/helloworld:predict'
data = json.dumps({"instances": x.tolist()})
headers = {"content-type": "application/json"}
run_prediction(data,headers,endpoint)

{
    "predictions": [[0.0]
    ]
}


{'predictions': [[0.0]]}